In [ ]:
import torch
from diffusers import LCMScheduler, AutoPipelineForText2Image

model_id = "stabilityai/stable-diffusion-xl-base-1.0"
adapter_id = "latent-consistency/lcm-lora-sdxl"

pipe = AutoPipelineForText2Image.from_pretrained(model_id, torch_dtype=torch.float16, variant="fp16")
pipe.scheduler = LCMScheduler.from_config(pipe.scheduler.config)
pipe.to("cuda")

# load and fuse lcm lora
pipe.load_lora_weights(adapter_id)
pipe.fuse_lora()


prompt = "Self-portrait oil painting, a beautiful cyborg with golden hair, 8k"

# disable guidance_scale by passing 0
image = pipe(prompt=prompt, num_inference_steps=4, guidance_scale=0).images[0]


In [ ]:
from pruna.algorithms.smasher_config import AutoSmasherConfig
from pruna.smash import smash

In [ ]:
from pruna.algorithms.smasher_config import SmasherConfig

smasher_config = SmasherConfig()
smasher_config['compiler'] = 'diffusers'
smasher_config['max_batch_size'] = 1
smasher_config['image_height'] = 1024
smasher_config['image_width'] = 1024
smasher_config['version'] = 'xl-1.0'
smasher_config['scheduler'] = "LCMScheduler"

In [ ]:
smashed_model = smash(
        model=pipe,
        data_module="LAION256",
        api_key='your-api-key',
        model_config=None,
        smasher_config=smasher_config,
        device='cuda',
    )

In [ ]:
%%time
smashed_model(prompt, num_inference_steps=4, image_height=1024, image_width=1024, guidance_scale=1.0)[0]

In [ ]:
%%time
pipe(prompt, num_inference_steps=4, guidance_scale=0).images[0]